# Carga de librerias

In [28]:
from elasticsearch import Elasticsearch
from json import dumps

es = Elasticsearch("http://127.0.0.1:9200")

print(f"Connected to ElasticSearch cluster `{es.info().body['cluster_name']}`")

Connected to ElasticSearch cluster `docker-cluster`


# Inserción de 1 millón de documentos

En esta sección se revisará la carga y el tamaño de almacenamiento de 1 millón de documentos indexados en la base de datos.

## Carga de documentos

La carga de documentos se llevará a cabo mediante el script de python ```load-documents-elastic.py``` localizado en el mismo directorio de este cuaderno. Para cargar la cantidad de datos en cuestión se ejecuta el siguiente comando:

```
python load-documents-elastic.py -n 1

```

**Nota:** es recomendable ejecutar el comando en una terminal aparte de este cuaderno, ya que se trata de una operación que dependiendo la cantidad de documentos, puede llegar a ocupar una considerable cantidad de memoria del sistema.

![Cargando 1 millón de documentos](../imagenes-de-soporte/elastic-insert/load_1M_documents_elastic.png)

Como se puede ver en la imagen superior, cargar 1 millón de documentos ha tomado un tiempo de **4 minutos y 26.16 segundos**.

Una vez cargados los documentos en la base de datos, podemos verificar la cantidad total de documentos en el índice ```extracto_cuenta_1m``` de la siguiente manera.

In [33]:
cnt = es.cat.count(index="extracto_cuenta_1m", format= "json")[0]['count']
print(f"Actualmente existen {cnt} documentos en el índice 'extracto_cuenta_1m'")

Actualmente existen 1000000 documentos en el índice 'extracto_cuenta_1m'


## Tamaño de almacenamiento de datos

Para obtener las estadísticas de uso de disco por parte del índice ```extracto_cuenta_1m``` en la base de datos ejecutaremos el siguiente comando:

In [35]:
r = dumps(es.indices.disk_usage(index="extracto_cuenta_1m", run_expensive_tasks=True)['extracto_cuenta_1m'], indent=1)
print(r[:500])
print("...")

{
 "store_size": "138.3mb",
 "store_size_in_bytes": 145089787,
 "all_fields": {
  "total": "137.5mb",
  "total_in_bytes": 144187007,
  "inverted_index": {
   "total": "19mb",
   "total_in_bytes": 19932341
  },
  "stored_fields": "87.7mb",
  "stored_fields_in_bytes": 91994865,
  "doc_values": "12.8mb",
  "doc_values_in_bytes": 13503952,
  "points": "15.9mb",
  "points_in_bytes": 16755849,
  "norms": "1.9mb",
  "norms_in_bytes": 2000000,
  "term_vectors": "0b",
  "term_vectors_in_bytes": 0,
  "knn
...


Extrayendo información importante de los datos recién obtenidos. El tamaño de almacenamiento de 1 documento de 1 millón de registros con un peso en bruto de 188.5 mb, **una vez comprimido en la base de datos es 138.3mb**. Esto nos da un radio de compresión de 1.36 y una reducción de tamaño en memoria del 26.59%.

![Cargando 1 millón de documentos](../imagenes-de-soporte/elastic-insert/storage_kibana_1M_documents.png)

------------------------

# Inserción de 10 millones de documentos

En esta sección se revisará la carga y el tamaño de almacenamiento de 10 millones de documentos indexados en la base de datos.

## Carga de documentos

La carga de documentos se llevará a cabo mediante el script de python ```load-documents-elastic.py``` localizado en el mismo directorio de este cuaderno. Para cargar la cantidad de datos en cuestión se ejecuta el siguiente comando:

```
python load-documents-elastic.py -n 10

```

**Nota:** es recomendable ejecutar el comando en una terminal aparte de este cuaderno, ya que se trata de una operación que dependiendo la cantidad de documentos, puede llegar a ocupar una considerable cantidad de memoria del sistema.

![Cargando 1 millón de documentos](../imagenes-de-soporte/elastic-insert/load_10M_documents_elastic.png)

Como se puede ver en la imagen superior, cargar 10 millones de documentos ha tomado un tiempo de **48 minutos y 52.61 segundos**.

Una vez cargados los documentos en la base de datos, podemos verificar la cantidad total de documentos en el índice ```extracto_cuenta_10m``` de la siguiente manera.

In [36]:
cnt = es.cat.count(index="extracto_cuenta_10m", format= "json")[0]['count']
print(f"Actualmente existen {cnt} documentos en el índice 'extracto_cuenta_10m'")

Actualmente existen 10000000 documentos en el índice 'extracto_cuenta_10m'


## Tamaño de almacenamiento de datos

Para obtener las estadísticas de uso de disco por parte del índice ```extracto_cuenta_10m``` en la base de datos ejecutaremos el siguiente comando:

In [40]:
r = dumps(es.indices.disk_usage(index="extracto_cuenta_10m", run_expensive_tasks=True)['extracto_cuenta_10m'], indent=1)
print(r[:500])
print("...")

{
 "store_size": "1.3gb",
 "store_size_in_bytes": 1440202391,
 "all_fields": {
  "total": "1.3gb",
  "total_in_bytes": 1438808499,
  "inverted_index": {
   "total": "191.3mb",
   "total_in_bytes": 200694994
  },
  "stored_fields": "876.7mb",
  "stored_fields_in_bytes": 919322081,
  "doc_values": "128.7mb",
  "doc_values_in_bytes": 135007131,
  "points": "156.1mb",
  "points_in_bytes": 163784293,
  "norms": "19mb",
  "norms_in_bytes": 20000000,
  "term_vectors": "0b",
  "term_vectors_in_bytes": 0
...


Extrayendo información importante de los datos recién obtenidos. El tamaño de almacenamiento de 10 documento de 1 millón de registros con un peso en bruto de 1.9 gb, **una vez comprimido en la base de datos es 1.34 gb**. Esto nos da un radio de compresión de 1.42 y una reducción de tamaño en memoria del 29.47%.

![Cargando 1 millón de documentos](../imagenes-de-soporte/elastic-insert/kibana_10M_documents.png)